In [ ]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

In [ ]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [ ]:
client.search_experiments()

In [ ]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.accuracy < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.accuracy ASC"]
)

In [ ]:
for run in runs:
    print(f"run id: {run.info.run_id}, accuracy: {run.data.metrics['accuracy']:.4f}")

Interacting with Model Registry

In [ ]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [ ]:
run_id = "45490009a7864814b3b4b0fdad5a8d52"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="project-health-stroke")

In [ ]:
client.search_registered_models()

Verify the models registered wrt its stages

In [ ]:
model_name = "project-health-stroke"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

Manually transition the stage of particular version of model

In [ ]:
model_version = 1
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

Update the registered models with its transition details wrt date and time

It is like a description

In [ ]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

Check the staged version code rmse is predicted good with the test data and move to production

we can move to prod and also can check the predictions

In [ ]:
'''
client.transition_model_version_stage(
    name=model_name,
    version=1,
    stage="Production",
    archive_existing_versions=True
)
'''